In [ ]:
import numpy as np
import nlopt
import math
import random
from scipy.linalg import block_diag
from numpy import *

In [ ]:
def symplectic_trace(A):
    """
    Compute symplectic trace of a 2×2 matrix A
    sTr(A) = sqrt(det(A))
    Returns 0 if det(A) < 0 (shouldn't happen with proper constraints)
    """
    det_A = np.linalg.det(A)
    if np.real(det_A) < 0:
        # This shouldn't happen if constraints are satisfied
        # Return 0 to signal invalid state
        print('NOOOOOOO')
        return 0.0
    return np.sqrt(np.real(det_A))

In [ ]:
def measurement(theta, psi, phi):
    """Return the measurement for specific angles

    Parameters:
        theta: theta angle in [0,pi]
        psi: psi angle in [0,pi]
        z: phi angle in [0,2pi]
    Returns:
        A 4x4 numpy array
    """
    u=np.matrix([math.cos(theta)*math.cos(psi-phi), math.cos(theta)*math.sin(psi-phi), math.sin(theta)*math.cos(psi), math.sin(theta)*math.sin(psi)])
    v=np.matrix([math.cos(theta)*math.cos(psi-phi),math.cos(theta)*math.sin(psi-phi),math.sin(theta)*math.cos(psi),math.sin(theta)*math.sin(psi)])
    return np.outer(u,v.T)

In [ ]:
def qmult_unit(n):
    """Generates a Haar-random unitary matrix of size n"""
    F = np.random.randn(n, n) + 1j * np.random.randn(n, n)
    Q, R = np.linalg.qr(F)
    return Q @ np.diag(np.exp(1j * np.random.rand(n) * 2 * np.pi))

def symp_orth(n):
    """Generates a symplectic orthogonal matrix of size n"""
    B = qmult_unit(n)
    Re = np.real(B)
    Im = np.imag(B)
    return np.block([[Re, -Im],
                    [Im,  Re]])

In [ ]:
def rand_rsymp(n,c):
    """Generate random symplectic matrix with specified symplectic eigenvalues.
    Parameters:
        n: dimension (2nx2n)
        c: symplectic eigenvalue(s) - either a scalar or n-vector
    Returns:
        A symplectic matrix
    """
    assert (len(c)==1 or len(c)==n),  (f"Expected vector component length for "
                                       f"symplectic matrix generation either 1 or {n}, got: {len(c)}")
    for scalar in c:
        assert scalar >= 1, f"Expected vector component for symplectic matrix generation >=, got: {scalar}"

    s = np.zeros(2*n)
    if(len(c)==1):
        s[0]=np.sqrt(c[0])
        s[1:n] = np.sort(1 + (s[0] - 1) * np.random.rand(n-1))[::-1]
        s[n:] = 1.0 / s[:n]
    else:
        c = np.sort(c)
        s[0:n] = c[::-1]
        s[n:2*n] = 1.0 / s[0:n]

    U=symp_orth(n)
    V=symp_orth(n)

    return U @ np.diag(s) @ V

In [ ]:
def randCM(entg = 1):
    """Searches for a random state with a specific entanglement level
    Parameters:
        entg: entanglement level
    Returns:
        A 4x4 matrix
    """
    int = 5
    rot = 10
    num = 300000

    P = np.matrix([[1,0,0,0],
                  [0,0,1,0],
                  [0,1,0,0],
                  [0,0,0,1]])

    x = 1
    y = 1.1

    a=1
    b=10

    for i in range(num):
        # generate thermal state CM
        g1 = np.matrix([[random.uniform(x,y),0],
                        [0,random.uniform(x,y)]]),
        g2 = block_diag(g1, g1)


        # generate random symplectic transformations with symplectic eigenvalues between 1 and 7.5
        S=rand_rsymp(2,(random.uniform(a,b),random.uniform(a,b)))
        # apply symplectic transformations to obtain a general CM

        g3 = S.transpose()@g2@S
        cm = P.transpose()@g3@P

        # calculate the logarithmic negativity
        det_11 = np.linalg.det(cm[0:2, 0:2])
        det_22 = np.linalg.det(cm[2:4, 2:4])
        det_12 = np.linalg.det(cm[0:2, 2:4])
        det_cm = np.linalg.det(cm)

        f = (0.5 * (det_11 + det_22) - det_12
             - np.sqrt((0.5 * (det_11 + det_22) - det_12)**2 - det_cm))

        EN = -0.5 * np.log2(f)
        EN1 = np.round(EN * rot) / rot

        if EN1 == entg/int:
            return cm
    return None

In [ ]:
def steering_detection(M_list, m_list, num_ops=14):
    """
    Detect steering using NLopt with specific constraints

    Constraints:
    - W ≥ 0 (PSD)
    - sTr(Z11) + sTr(Z22) ≥ 0.5
    """
    n_vars = num_ops

    # Objective function: w·m
    def objective(w, grad):
        obj = np.dot(w, m_list)

        if grad.size > 0:
            grad[:] = m_list

        return float(obj)

    # Constraint 1: W = Σ w_k M_k ≥ 0
    def constraint_W_psd(w, grad):
        W = np.sum([w[k] * M_list[k] for k in range(num_ops)], axis=0)

        # Check PSD: minimum eigenvalue ≥ 0
        eigvals = np.linalg.eigvalsh(W)
        min_eigval = np.min(np.real(eigvals))

        if grad.size > 0:
            # Numerical gradient approximation
            eps = 1e-8
            for i in range(num_ops):
                w_plus = w.copy()
                w_plus[i] += eps
                W_plus = np.sum([w_plus[k] * M_list[k] for k in range(num_ops)], axis=0)
                eigvals_plus = np.linalg.eigvalsh(W_plus)
                min_eigval_plus = np.min(np.real(eigvals_plus))
                grad[i] = (min_eigval_plus - min_eigval) / eps

        return float(min_eigval)
    # Constraint 2: det(Z11) ≥ 0
    def constraint_det_Z11(w, grad):
        W = np.sum([w[k] * M_list[k] for k in range(num_ops)], axis=0)
        Z11 = W[0:2, 0:2]
        det_Z11 = np.real(np.linalg.det(Z11))

        if grad.size > 0:
            eps = 1e-8
            for i in range(num_ops):
                w_plus = w.copy()
                w_plus[i] += eps
                W_plus = np.sum([w_plus[k] * M_list[k] for k in range(num_ops)], axis=0)
                Z11_plus = W_plus[0:2, 0:2]
                det_plus = np.real(np.linalg.det(Z11_plus))
                grad[i] = (det_plus - det_Z11) / eps

        return float(det_Z11)

    # Constraint 3: det(Z22) ≥ 0
    def constraint_det_Z22(w, grad):
        W = np.sum([w[k] * M_list[k] for k in range(num_ops)], axis=0)
        Z22 = W[2:4, 2:4]
        det_Z22 = np.real(np.linalg.det(Z22))

        if grad.size > 0:
            eps = 1e-8
            for i in range(num_ops):
                w_plus = w.copy()
                w_plus[i] += eps
                W_plus = np.sum([w_plus[k] * M_list[k] for k in range(num_ops)], axis=0)
                Z22_plus = W_plus[0:2, 0:2]
                det_plus = np.real(np.linalg.det(Z22_plus))
                grad[i] = (det_plus - det_Z22) / eps

        return float(det_Z22)

    # Constraint 2: sTr(Z11) + sTr(Z22) ≥ 0.5
    def constraint_symplectic_trace(w, grad):
        W = np.sum([w[k] * M_list[k] for k in range(num_ops)], axis=0)

        # Extract 2×2 blocks
        Z11 = W[0:2, 0:2]
        Z22 = W[2:4, 2:4]

        det_Z11 = np.real(np.linalg.det(Z11))
        det_Z22 = np.real(np.linalg.det(Z22))

        # If check if determinants are negative
        if det_Z11 < 0 or det_Z22 < 0:
            constraint_val = np.min([det_Z11, det_Z22])
            if grad.size > 0:
                grad[:] = 0  # Don't compute gradient in invalid region
            return float(constraint_val)
        else:
            sTr_Z11 = np.sqrt(det_Z11)
            sTr_Z22 = np.sqrt(det_Z22)
            constraint_val = sTr_Z11 + sTr_Z22 - 0.5

        if grad.size > 0:
            # Numerical gradient approximation
            eps = 1e-8
            for i in range(num_ops):
                w_plus = w.copy()
                w_plus[i] += eps
                W_plus = np.sum([w_plus[k] * M_list[k] for k in range(num_ops)], axis=0)
                Z11_plus = W_plus[0:2, 0:2]
                Z22_plus = W_plus[2:4, 2:4]

                det_Z11_plus = float(np.real(np.linalg.det(Z11_plus)))
                det_Z22_plus = float(np.real(np.linalg.det(Z22_plus)))

                # Compute gradient only if both current and perturbed are valid
                if det_Z11 >= 0 and det_Z22 >= 0 and det_Z11_plus >= 0 and det_Z22_plus >= 0:
                    sTr_current = np.sqrt(det_Z11) + np.sqrt(det_Z22)
                    sTr_plus = np.sqrt(det_Z11_plus) + np.sqrt(det_Z22_plus)
                    grad[i] = (sTr_plus - sTr_current) / eps
                elif det_Z11 < 0 or det_Z22 < 0:
                    current_val = np.min([det_Z11, det_Z22])
                    plus_val = np.min([det_Z11_plus, det_Z22_plus])
                    grad[i] = (plus_val - current_val) / eps
                else:
                    grad[i] = 0

        return float(constraint_val)

    # Set up NLopt optimizer; gradient based (for now)
    opt = nlopt.opt(nlopt.LD_SLSQP, n_vars)

    # Set objective (minimize)
    opt.set_min_objective(objective)

    # Add inequality constraints (≥ 0)
    opt.add_inequality_constraint(constraint_W_psd, 1e-8)
    opt.add_inequality_constraint(constraint_det_Z11, 1e-6)  # Enforce det ≥ 0
    opt.add_inequality_constraint(constraint_det_Z22, 1e-6)  # Enforce det ≥ 0
    opt.add_inequality_constraint(constraint_symplectic_trace, 1e-8)

    # Set bounds on weights (optional)
    opt.set_lower_bounds(-10 * np.ones(n_vars))
    opt.set_upper_bounds(10 * np.ones(n_vars))

    # Set tolerance
    opt.set_xtol_rel(1e-6)
    opt.set_maxeval(10000)

    # Initial guess
    w0 = np.ones(n_vars) / num_ops  # Equal weights

    # Optimize
    try:
        w_opt = opt.optimize(w0)
        min_value = opt.last_optimum_value()

        return min_value, w_opt
    except Exception as e:
        print(f"Optimization failed: {e}")
        return np.inf, None


In [ ]:
def check_constraints(w_opt, M_list, min_val, num_ops=14):
    """
    Sanity check: verify all constraints are satisfied

    Returns:
        dict with constraint values and whether they're satisfied
    """
    W = np.sum([w_opt[k] * M_list[k] for k in range(num_ops)], axis=0)

    # Check 1: W ≥ 0 (PSD)
    eigvals = np.linalg.eigvalsh(W)
    min_eigval = np.min(np.real(eigvals))
    W_psd = min_eigval >= -1e-6  # Small tolerance for numerical errors

    # Check 2: sTr(Z11) + sTr(Z22) ≥ 0.5
    Z11 = W[0:2, 0:2]
    Z22 = W[2:4, 2:4]

    det_Z11 = np.real(np.linalg.det(Z11))
    det_Z22 = np.real(np.linalg.det(Z22))

    if det_Z11 >= 0 and det_Z22 >= 0:
        sTr_sum = np.sqrt(det_Z11) + np.sqrt(det_Z22)
        sTr_satisfied = sTr_sum >= 0.5 - 1e-6
    else:
        sTr_sum = None
        sTr_satisfied = False

    results = {
        'min_eigval_W': min_eigval,
        'W_is_PSD': W_psd,
        'det_Z11': det_Z11,
        'det_Z22': det_Z22,
        'sTr_sum': sTr_sum,
        'sTr_satisfied': sTr_satisfied,
        'objective_value': min_val,
        'all_constraints_ok': W_psd and sTr_satisfied
    }

    print("=== Constraint Check ===")
    print(f"Objective (w·m): {min_val:.6f}")
    print(f"Min eigenvalue of W: {min_eigval:.6f} {'✓' if W_psd else '✗'}")
    print(f"det(Z11): {det_Z11:.6f}")
    print(f"det(Z22): {det_Z22:.6f}")
    if sTr_sum is not None:
        print(f"sTr(Z11) + sTr(Z22): {sTr_sum:.6f} {'✓' if sTr_satisfied else '✗'} (need ≥ 0.5)")
    else:
        print(f"sTr constraint: ✗ (negative determinants)")
    print(f"All constraints satisfied: {'YES ✓' if results['all_constraints_ok'] else 'NO ✗'}")

    return results

In [ ]:
# Main detection loop
num_ops = 14
num_rep = 2
num_ent = 1

nn = np.zeros((num_ops, num_ent), dtype=int)

for kdx in range(1, num_ent + 1):
    n = np.zeros(num_ops, dtype=int)

    for jdx in range(num_rep):
        # Generate state
        g = randCM(kdx)

        # Generate random measurements
        theta = np.pi * np.random.rand(num_ops)
        psi = np.pi * np.random.rand(num_ops)
        phi = 2 * np.pi * np.random.rand(num_ops)

        M_list = []
        m_list = []

        for idx in range(num_ops):
            M = measurement(theta[idx], psi[idx], phi[idx])
            M_list.append(M)
            m_list.append(np.real(np.trace(M @ g)))

        # Run optimization
        min_val, w_opt = steering_detection(M_list, m_list, num_ops)

        print(f"kdx={kdx}, jdx={jdx}, min_val={min_val:.6f}")

        # Check if steering detected
        if min_val < 1.0:
            print(f"  → Steering detected!")
            n[0] += 1  # Record success
            break
            check_constraints(w_opt, M_list, min_val, num_ops)

    nn[:, kdx-1] = n

# Save results
np.savetxt('gen1.txt', nn, fmt='%d')
print("\nResults saved to gen1.txt")
print(nn)
